In [1]:
import pandas as pd
import numpy as np
import os
import glob
import shutil
import xml.etree.ElementTree as ET
import random

In [2]:
path ='../data/annotations.xml'
dataset = []

for anno in glob.glob(path):
    tree = ET.parse(anno)
    root = tree.getroot()
    
    for image_elem in root.iter("image"):
        image_attrs = image_elem.attrib
        image_data = {
            "filename": image_attrs['name'],
            "width": float(image_attrs['width']),
            "height": float(image_attrs['height']),
            "boxes": []
        }
        
        for box_elem in image_elem.iter("box"):
            box_attrs = box_elem.attrib
            box_data = {
                "label": box_attrs['label'],
                "occluded": int(box_attrs['occluded']),
                "xtl": float(box_attrs['xtl']),
                "ytl": float(box_attrs['ytl']),
                "xbr": float(box_attrs['xbr']),
                "ybr": float(box_attrs['ybr'])
            }
            image_data["boxes"].append(box_data)
        
        dataset.append(image_data)

In [3]:
df = pd.DataFrame(dataset)
df.head()

,filename,width,height,boxes
0,images/0.png,1200.0,800.0,"[{'label': 'strawberry', 'occluded': 0, 'xtl':..."
1,images/1.png,1600.0,900.0,"[{'label': 'strawberry', 'occluded': 1, 'xtl':..."
2,images/10.png,920.0,425.0,"[{'label': 'strawberry', 'occluded': 1, 'xtl':..."
3,images/11.png,640.0,480.0,"[{'label': 'strawberry', 'occluded': 0, 'xtl':..."
4,images/12.png,1920.0,1080.0,"[{'label': 'strawberry', 'occluded': 0, 'xtl':..."


In [4]:
df_=pd.DataFrame()
for i in range(len(df)):
    boxes=df.loc[i,'boxes']
    for box in boxes:
        a=df.loc[i,'filename']
        b=df.loc[i,'width']
        c=df.loc[i,'height']
        d=box['xtl']
        e=box['ytl']
        f=box['xbr']
        g=box['ybr']
        add_df=pd.DataFrame([[a,b,c,d,e,f,g]])
        df_=pd.concat([df_,add_df],axis=0)

df_.columns=['filename','width','height','xtl','ytl','xbr','ybr']
df_

,filename,width,height,xtl,ytl,xbr,ybr
0,images/0.png,1200.0,800.0,368.18,292.79,460.65,454.46
0,images/0.png,1200.0,800.0,423.70,268.56,537.50,471.16
0,images/0.png,1200.0,800.0,538.93,338.40,666.51,509.38
0,images/0.png,1200.0,800.0,642.29,416.47,789.86,616.35
0,images/0.png,1200.0,800.0,734.64,416.40,837.78,597.06
...,...,...,...,...,...,...,...
0,images/9.png,1600.0,1200.0,1208.45,713.03,1386.04,922.00
0,images/9.png,1600.0,1200.0,126.12,207.98,277.09,431.25
0,images/9.png,1600.0,1200.0,136.68,555.56,416.67,900.57
0,images/9.png,1600.0,1200.0,261.24,523.59,490.52,805.78


In [5]:
df_['label']=0
df_['Xcent']=(df_['xtl']+df_['xbr'])/(2*df_['width'])
df_['Ycent']=(df_['ytl']+df_['ybr'])/(2*df_['height'])
df_['boxW']=(df_['xbr']-df_['xtl'])/df_['width']
df_['boxH']=(df_['ybr']-df_['ytl'])/df_['height']
df_

,filename,width,height,xtl,ytl,xbr,ybr,label,Xcent,Ycent,boxW,boxH
0,images/0.png,1200.0,800.0,368.18,292.79,460.65,454.46,0,0.345346,0.467031,0.077058,0.202087
0,images/0.png,1200.0,800.0,423.70,268.56,537.50,471.16,0,0.400500,0.462325,0.094833,0.253250
0,images/0.png,1200.0,800.0,538.93,338.40,666.51,509.38,0,0.502267,0.529863,0.106317,0.213725
0,images/0.png,1200.0,800.0,642.29,416.47,789.86,616.35,0,0.596729,0.645513,0.122975,0.249850
0,images/0.png,1200.0,800.0,734.64,416.40,837.78,597.06,0,0.655175,0.633412,0.085950,0.225825
...,...,...,...,...,...,...,...,...,...,...,...,...
0,images/9.png,1600.0,1200.0,1208.45,713.03,1386.04,922.00,0,0.810778,0.681262,0.110994,0.174142
0,images/9.png,1600.0,1200.0,126.12,207.98,277.09,431.25,0,0.126003,0.266346,0.094356,0.186058
0,images/9.png,1600.0,1200.0,136.68,555.56,416.67,900.57,0,0.172922,0.606721,0.174994,0.287508
0,images/9.png,1600.0,1200.0,261.24,523.59,490.52,805.78,0,0.234925,0.553904,0.143300,0.235158


In [ ]:
df_.to_csv('../data/annotation.csv',index=False)

In [7]:
files=df_['filename'].unique().tolist()
for file in files:
    datai=df_[df_['filename']==file].iloc[:,7:]
    name=file.split('/')[-1][0:-4]
    fmt = ['%d'] + ['%f'] * (datai.shape[1]-1)
    np.savetxt("../data/labels/"+name+".txt", np.array(datai), fmt=fmt, delimiter="\t")

In [8]:
train_path = '../data/train'
train_img_path = '../data/train/images'
train_labels_path = '../data/train/labels'
test_path = '../data/test'
test_img_path = '../data/test/images'
test_labels_path = '../data/test/labels'
valid_path = '../data/valid'
valid_img_path = '../data/valid/images'
valid_labels_path = '../data/valid/labels'

In [9]:
for p in [train_path, train_img_path, train_labels_path, test_path, test_img_path, test_labels_path, valid_path, valid_img_path, valid_labels_path]:
    os.makedirs(p, exist_ok=True)

In [10]:
img_path = '../data/images'
paths=[]
for dirname, _, filenames in os.walk(img_path):
    for filename in filenames:
        paths+=[(os.path.join(dirname, filename))]
paths=random.sample(paths,len(paths))
print(paths[0:5])

['../data/images\\7.png', '../data/images\\19.png', '../data/images\\39.png', '../data/images\\16.png', '../data/images\\2.png']


In [11]:
for i, path in enumerate(paths):
    try:
        # Determine the destination directories based on the index
        if i < 25:
            dest_dir_img = train_img_path
            dest_dir_label = train_labels_path
        elif i < 35:
            dest_dir_img = valid_img_path
            dest_dir_label = valid_labels_path
        else:
            dest_dir_img = test_img_path
            dest_dir_label = test_labels_path

        shutil.copy(path, dest_dir_img)

        base_filename = os.path.basename(path)
        filename_without_ext = os.path.splitext(base_filename)[0]
        label_file = filename_without_ext + '.txt'
        label_source_path = os.path.join('../data/labels', label_file)

        if os.path.exists(label_source_path):
            shutil.copy(label_source_path, dest_dir_label)
        else:
            print(f"Warning: Label file not found and was not copied: {label_source_path}")

    except FileNotFoundError as e:
        print(f"Error: File not found - {e}. Skipping this iteration.")
    except Exception as e:
        print(f"An unexpected error occurred at index {i} for path {path}: {e}")

In [ ]:
import torch
from ultralytics import YOLO
from PIL import Image

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\venna\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [14]:
model = YOLO("yolo11n.pt")

100%|██████████| 5.35M/5.35M [00:01<00:00, 4.03MB/s]


In [17]:
data_yml = '../data/dataset.yml'
train_results = model.train(
    data='../data/dataset.yml',  # Path to dataset configuration file
    epochs=100,  # Number of training epochs
    imgsz=640,  # Image size for training
    device="0",  # Device to run on (e.g., 'cpu', 0, [0,1,2,3])
    workers=0
)

Ultralytics 8.3.160  Python-3.11.7 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/dataset.yml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=

100%|██████████| 755k/755k [00:00<00:00, 2.14MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning C:\Users\venna\Desktop\deep_learning_practise\computer_vision\object_detection_segmentation\ripped_strawberry\data\train\labels... 35 images, 0 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<00:00, 447.32it/s]

train: New cache created: C:\Users\venna\Desktop\deep_learning_practise\computer_vision\object_detection_segmentation\ripped_strawberry\data\train\labels.cache
val: Fast image access  (ping: 0.30.2 ms, read: 92.425.3 MB/s, size: 1125.3 KB)


...

    100/100      2.33G     0.7536     0.6639      0.946         31        640: 100%|██████████| 3/3 [00:05<00:00,  1.83s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<00:00,  3.49s/it]

                   all         17        217      0.931      0.862      0.929      0.683



100 epochs completed in 0.177 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 5.5MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.160  Python-3.11.7 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.64s/it]


                   all         17        217      0.936      0.862      0.934      0.698
Speed: 1.7ms preprocess, 10.9ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to runs\detect\train2
MLflow: results logged to runs\mlflow
MLflow: disable with 'yolo settings mlflow=False'


In [18]:
metrics = model.val()

Ultralytics 8.3.160  Python-3.11.7 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 1572.9220.7 MB/s, size: 772.4 KB)


val: Scanning C:\Users\venna\Desktop\deep_learning_practise\computer_vision\object_detection_segmentation\ripped_strawberry\data\valid\labels.cache... 17 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17/17 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.16s/it]


                   all         17        217      0.933      0.862      0.934      0.694
Speed: 0.6ms preprocess, 79.8ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to runs\detect\train22


In [24]:
results = model("../images.jpg")  # Predict on an image
results[0].show()  # Display results


image 1/1 c:\Users\venna\Desktop\deep_learning_practise\computer_vision\object_detection_segmentation\ripped_strawberry\notebook\..\images.jpg: 448x640 1 ripped_strawberry, 27.9ms
Speed: 2.3ms preprocess, 27.9ms inference, 5.3ms postprocess per image at shape (1, 3, 448, 640)
